In [1]:
import os

In [2]:
%pwd

'e:\\Projects\\MLproject-2_Kidney-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Projects\\MLproject-2_Kidney-Disease-Classification'

In [5]:
import tensorflow as tf

In [7]:
# model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/shahin509/MLproject-2_Kidney-Disease-Classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config


In [ ]:
# !pip install keras

In [11]:
from urllib.parse import urlparse
from pathlib import Path
import mlflow
import mlflow.keras
import dagshub

In [12]:
# Initialize DagsHub MLflow tracking
dagshub.init(repo_owner='shahin509', repo_name='MLproject-2_Kidney-Disease-Classification', mlflow=True)

[2024-11-27 12:20:54,512: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as shahin509

[2024-11-27 12:20:54,579: INFO: helpers: Accessing as shahin509]
[2024-11-27 12:20:55,302: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/shahin509/MLproject-2_Kidney-Disease-Classification "HTTP/1.1 200 OK"]
[2024-11-27 12:20:55,994: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "shahin509/MLproject-2_Kidney-Disease-Classification"

[2024-11-27 12:20:56,001: INFO: helpers: Initialized MLflow to track repo "shahin509/MLproject-2_Kidney-Disease-Classification"]


Repository shahin509/MLproject-2_Kidney-Disease-Classification initialized!

[2024-11-27 12:20:56,003: INFO: helpers: Repository shahin509/MLproject-2_Kidney-Disease-Classification initialized!]


In [13]:
from cnnClassifier import logger

class Evaluation:
    def __init__(self, config):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        try:
            # Start the MLflow tracking
            with mlflow.start_run():
                # Log parameters and metrics
                mlflow.log_params(self.config.all_params)
                mlflow.log_metrics(
                    {"loss": self.score[0], "accuracy": self.score[1]}
                )

                # Determine the tracking URI type
                tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

                # Log and optionally register the model
                if tracking_url_type_store != "file":
                    # Register the model in the MLflow Model Registry
                    mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
                else:
                    mlflow.keras.log_model(self.model, "model")
    
        except Exception as e:
            # Log the exception and re-raise
            logger.exception("An error occurred during logging into MLflow.")
            raise e

In [14]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-11-27 12:21:08,027: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-27 12:21:08,066: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-27 12:21:08,085: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 26s 2s/step - loss: 0.0817 - accuracy: 0.9640
[2024-11-27 12:21:38,055: INFO: common: json file saved at: scores.json]


2024/11/27 12:21:42 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-11-27 12:21:44,206: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\USER\AppData\Local\Temp\tmphz8jhl8l\model\data\model\assets
[2024-11-27 12:21:45,243: INFO: builder_impl: Assets written to: C:\Users\USER\AppData\Local\Temp\tmphz8jhl8l\model\data\model\assets]


e:\Projects\MLproject-2_Kidney-Disease-Classification\kidney\lib\site-packages\_distutils_hack\__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/11/27 12:24:03 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
